# 0. Ferromagnetic phase and paramagnetic phase classification

In this notebook, we attempt to classify the  ferromagnetic phase and the  paramagnetic phase. 

The Ising model is a mathematical model of ferromagnetism in statistical mechanics. The model consists of discrete variables that represent magnetic dipole moments of atomic "spins" that can be in one of two states (+1 or −1).

Below a certain temperature called the Curie temperature, $T_c$, most of the spins are parallel, which results in a uniform overall state characterised by $+1$ or $-1$. This phase is called the ferromagnetic phase. Above $T_c$, magnetic dipole moments of half of the atomic "spins" yield $+1$ while the other half yields $-1$. This phase is called the paramagnetic phase. 

This work aims to classify two different types of magnetic phases: a low-temperature ferromagnetic phase and a high-temperature paramagnetic phase. The details can be found in [Machine learning phases of matter](https://doi.org/10.1038/nphys4035). 

In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# check version
print('Using TensorFlow v%s' % tf.__version__)
acc_str = 'accuracy' if tf.__version__[:2] == '2.' else 'acc'

# helpers
import h5py
import numpy as np
import matplotlib.pyplot as plt
from os.path import join

# need some certainty in data processing
np.random.seed(1234)
tf.random.set_seed(1234)


## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: data_path = f.read().splitlines()[0]

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {project_id} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

# mount the bucket
echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
apt -qq update
apt -qq install gcsfuse
gcloud config set project $3
mkdir $2
gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 $4 $2

---

# 1. Load the dataset

### Read raw data

The data is stored in 'Physics/ising_data.npz', containing 26000, gray-scale, 32 × 32 images of the magnetic phases. We will use 20000 images to train neural networks and the rest of them to evaluate the neural networks.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
f = np.load(join(data_path, 'Physics/ising_data.npz'))
n_train = 20000

x_train, x_test = f["C"][:n_train], f["C"][n_train:]
T_train, T_test = f["T"][:n_train], f["T"][n_train:]
```
    
</p>
</details>

In [ ]:
# define image size
IMG_HEIGHT = 32
IMG_WIDTH = 32
N_CHANNELS = 1
N_CLASSES = 2

### normalize data
x_train = (x_train + 1)/2.
x_test = (x_test + 1)/2.
x_train = x_train.reshape(-1, 32, 32, 1)
x_test = x_test.reshape(-1, 32, 32, 1)

Tc = 2.27
y_train, y_test = T_train > Tc, T_test > Tc

string_labels = ['paramagnetic','ferromagnetic']

# print
print("Number of training data: %d" % len(x_train))
print("Number of test data: %d" % len(x_test))
print("Image pixels: %s" % str(x_train[0, :, :, 0].shape))
print("Number of channels: %s" % str(x_train.shape[-1]))

In [ ]:
# function to plot an image in a subplot
def subplot_image(image, label, nrows=1, ncols=1, iplot=0, label2='', label2_color='r'):
    plt.subplot(nrows, ncols, iplot + 1)
    plt.imshow(image.squeeze(), cmap=plt.cm.binary)
    plt.xlabel(label, c='k', fontsize=8)
    plt.title(label2, c=label2_color, fontsize=8, y=-0.33)
    plt.xticks([])
    plt.yticks([])
    
# ramdomly plot some images and their labels
nrows = 4
ncols = 8
plt.figure(dpi=100, figsize=(ncols * 2, nrows * 2.2))
for iplot, idata in enumerate(np.random.choice(len(y_train), nrows * ncols)):
    label = "%d: %s" % (y_train[idata], string_labels[y_train[idata]])
    subplot_image(x_train[idata], label, nrows, ncols, iplot)
plt.show()

---

# 2. Build the network

The task is to build and train a CNN to solve this binary classification problem. 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same',
                 input_shape=(32, 32, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# print summary
model.summary()
```
    
</p>
</details>

###  Compile the model

We can add the following metrics to see how the network performs for the two classes:

* `TruePositives`: number of right predictions for paramagnetic
* `FalsePositives`: number of wrong predictions for paramagnetic
* `TrueNegatives`: number of right predictions for ferromagnetic
* `FalseNegatives`: number of wrong predictions for ferromagnetic

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics= ['accuracy',
                        keras.metrics.TruePositives(name='true_positives'),
                        keras.metrics.FalsePositives(name='false_positives'),
                        keras.metrics.TrueNegatives(name='true_negatives'),
                        keras.metrics.FalseNegatives(name='false_negatives')])
```
    
</p>
</details>


### Train the model

Using the suggested architecture, we can achieve an accuracy greater than 97% with 2 epochs. 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
stopper=EarlyStopping( monitor ="val_loss", min_delta=0.0,verbose=1,
                      mode="min",
                      restore_best_weights=True,patience=2)

training_history = model.fit(x_train, y_train, validation_data=(x_test, y_test), 
                             epochs=10, batch_size=64,callbacks=[stopper])

# print final values of metrics for validation data
print('Right for Paramagnetic: %d' % training_history.history['val_true_positives'][-1])
print('Wrong for Paramagnetic: %d' % training_history.history['val_false_positives'][-1])
print('Right for Ferromagnetic: %d' % training_history.history['val_true_negatives'][-1])
print('Wrong for Ferromagnetic: %d' % training_history.history['val_false_negatives'][-1])
```
    
</p>
</details>

---

# 3. Analyse results 

### Plot training history

For convenience, we define a function to plot a training history:

In [ ]:
# a function to plot training history
def plot_history(training_history):
    # plot accuracy
    plt.figure(dpi=100, figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(training_history.history[acc_str], label='Accuracy on training data')
    plt.plot(training_history.history['val_' + acc_str], label='Accuracy on test data')
    plt.legend()
    plt.title("Accuracy")

    # plot loss
    plt.subplot(1, 2, 2)
    plt.plot(training_history.history['loss'], label='Loss on training data')
    plt.plot(training_history.history['val_loss'], label='Loss on test data')
    plt.legend()
    plt.title("Loss")
    plt.show()

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# plot training history
plot_history(training_history)
```
    
</p>
</details>

# 4. Exercises

* Change some hyperparameters in `model.compile()` and `model.fit()` to see their effects (see reference of [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model)); 
* Change the architeture and activation functions of neural networks to improve the accuracy